<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/kaggle_predictFS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
    %tensorflow_version 2.x
except:
    print("Note: not using Google CoLab")
    COLAB = False

In [0]:
import os
os.chdir('/content/drive/My Drive/colab')
!cp kaggle.json /content

In [0]:
os.chdir('/content/')
# kaggle_Predint_Future_Sales
!pip install kaggle

In [0]:
os.chdir('/content')
!ls -1ha kaggle.json

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
import os
os.mkdir('/content/predFS/')
os.chdir('/content/predFS/')

In [0]:
!kaggle competitions download -c competitive-data-science-predict-future-sales

In [0]:
os.chdir('/content/predFS/')
!ls

In [0]:
os.mkdir('/content/predFS/input')
!mv item_categories.csv /content/predFS/input
!mv sales_train.csv.zip /content/predFS/input
!mv shops.csv /content/predFS/input
!mv items.csv.zip /content/predFS/input
!mv sample_submission.csv.zip /content/predFS/input
!mv test.csv.zip /content/predFS/input

In [0]:
os.chdir('/content/predFS/input')
!unzip sales_train.csv.zip 
!unzip items.csv.zip
!unzip sample_submission.csv.zip
!unzip test.csv.zip


In [0]:
#https://www.kaggle.com/dlarionov/feature-engineering-xgboost

# feature engineering
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc
import pickle
sys.version_info

In [0]:
items = pd.read_csv('../input/items.csv')
shops = pd.read_csv('../input/shops.csv')
cats = pd.read_csv('../input/item_categories.csv')
train = pd.read_csv('../input/sales_train.csv')
# set index to ID to avoid droping it later
test  = pd.read_csv('../input/test.csv').set_index('ID')

**Data Field**

**ID** - an Id that represents a (Shop, Item) tuple within the test set

**shop_id** - unique identifier of a shop

**item_id** - unique identifier of a product

**item_category_id** - unique identifier of item category

**item_cnt_day** - number of products sold. You are predicting a monthly amount of this measure

**item_price** - current price of an item

**date** - date in format dd/mm/yyyy

**date_block_num** - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

**item_name** - name of item

**shop_name** - name of shop

**item_category_name** - name of item category

In [0]:
train.head()

In [0]:
# outlier 찾기
plt.figure(figsize=(10,4)) # 그래프 판 사이즈
plt.xlim(-100, 3000) # x 좌표 설정
sns.boxplot(x=train.item_cnt_day) # 어떤 샵의 어떤 제품의 어떤 날짜 1일 판매량 박스 플롯으로

plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1) # x 좌표 설정, 가격 최소값과 최대값을 직접 입력
sns.boxplot(x=train.item_price) # 제품 가격 박스 플롯

In [0]:
#outlier 제거
train = train[train.item_price<100000]  # 가격 100000 미만
train = train[train.item_cnt_day<1001]  # 하루 판매량 1000 이하

In [0]:
# 결측값 채우기
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
# 가격 외 위 조건의 항목 중 가격이 음의 값이 존재하며, 이를 같은 조건을 가진 다른 케이스들의 중앙값(median)으로 채움
train.loc[train.item_price<0, 'item_price'] = median 
# loc 다음은 행 값, 열 값을 줄 수 있음. 이 케이스의 경우 가격이 음인 케이스는 하나이므로 행조건에 별다른 조건을 걸지 않음

In [0]:
cats[:20]

In [0]:
# 같은 가게를 다르게 입력한 케이스 통합, 단순 데이터 중복이 아닌 내용적 중복을 잡은 것이지만 그냥 스킬적으로만 알자
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [0]:
# 신규 특징(도시명, 서브카테고리 명) 생성
# 작성자는 가게앞 명칭이 도시명으로 되어 있다는 사실에 착안하여 도시명 생성
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
#샵 이름의 도시명 정제
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0]) 
# shop_name을 ' '으로 스플릿하여 ['도시이름', '샵이름']의 pd.Series가 나오며, map(lambda)의 x는 map앞을 받아 함수처리
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
# 데이터 정제로 보임(특수문자 제거)
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
# sklearn의 함수로 괄호안 이터레이션에 고유값 별 라벨을 만들어주고 해당 값에 따라 라벨을 붙여줌
shops = shops[['shop_id','city_code']]
# shops 테이블 간소화, 최종적으로 가게 id와 도시이름 code만 남김
# 가게명-도시명 관계 확인 -> 앞부분을 이용한 도시명 추출 -> 도시명 정제 -> 도시명 코드화 -> shop_id, city_code로 정리

# 상위 카테고리(제품 종류)와 하위 카테고리(브랜드? 종류) 구분
cats['split'] = cats['item_category_name'].str.split('-') # -로 연결된 두 개를 분리하여 DF로 남김
cats['type'] = cats['split'].map(lambda x: x[0].strip()) # 제품 유형을 type으로 
cats['type_code'] = LabelEncoder().fit_transform(cats['type']) # 제품 유형 라벨 붙이기
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())  # 브랜드가 없으면 유형으로
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])# 라벨링
cats = cats[['item_category_id','type_code', 'subtype_code']] # id와 코드들로만 테이블 정리

items.drop(['item_name'], axis=1, inplace=True) # 아이템 id와 카테고리 id만 남김

In [0]:
# 상위 카테고리(제품 종류)와 하위 카테고리(브랜드? 종류) 구분
cats['split'] = cats['item_category_name'].str.split('-') # -로 연결된 두 개를 분리하여 DF로 남김
cats['type'] = cats['split'].map(lambda x: x[0].strip()) # 제품 유형을 type으로 
cats['type_code'] = LabelEncoder().fit_transform(cats['type']) # 제품 유형 라벨 붙이기
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())  # 브랜드가 없으면 유형으로
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])# 라벨링
cats = cats[['item_category_id','type_code', 'subtype_code']] # id와 코드들로만 테이블 정리

items.drop(['item_name'], axis=1, inplace=True) # 아이템 id와 카테고리 id만 남김

In [0]:
#월별 판매량으로 정리
len(list(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))), len(list(set(test.item_id))), len(test)
#test 셋에만 존재하는 아이템 종류 수, 테스트에 있는 아이템 종류 총 수, 테스트 총 케이스 수

In [0]:
ts = time.time() # 셀 시작 시간
matrix = [] 
cols = ['date_block_num','shop_id','item_id']
for i in range(34): #훈련 세트의 날짜 블록이 34개
    sales = train[train.date_block_num==i] # 날짜 블록별로 DF 생성
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
      # len(i)와 sales.shop_id.unique(), sales.item_id.unique()의 카테시안 곱이 이뤄지며,
      # (1 * len(sales.shop_id.unique()), len(sales.item_id.unique()),3)의 리스트 생성 후 리스트에 더해진다.
matrix = pd.DataFrame(np.vstack(matrix), columns=cols) 
# vstack은 가장 작은 리스트 단위를 쌓아주는 것으로 리스트 내 원소 숫자는 같지만 리스트들의 숫자가 케이스별로 다를 경우
# vstack류를 사용하여 정렬시킬 수 있을 것
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8) # 정수 타입 변환
matrix['shop_id'] = matrix['shop_id'].astype(np.int8) # 정수 타입 변환
matrix['item_id'] = matrix['item_id'].astype(np.int16)  # 정수 타입 변환 
matrix.sort_values(cols,inplace=True) 
time.time() - ts

In [0]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']

In [0]:
ts = time.time()
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
# 월 블록 - 샵 id - 아이템 id로 테이블을 만들고 하루 판매량의 합을 값으로 설정
group.columns = ['item_cnt_month'] # 값 열을 월별 판매량으로 명명(date_block은 월 단위 순서임)
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left') # 모든 케이스가 월단위 판매량을 가지게 함
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0) 
                                .clip(0,20) # NB clip target here # 최대-최소 바운더리
                                .astype(np.float16))
time.time() - ts